In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

In [68]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '3users/user0001.csv'))

In [72]:
user31_data['session_id'] = (user31_data.index)/10
user31_data.head(20)

,timestamp,site,session_id
0,2013-11-15 09:28:17,vk.com,0
1,2013-11-15 09:33:04,oracle.com,0
2,2013-11-15 09:52:48,oracle.com,0
3,2013-11-15 11:37:26,geo.mozilla.org,0
4,2013-11-15 11:40:32,oracle.com,0
5,2013-11-15 11:40:34,google.com,0
6,2013-11-15 11:40:35,accounts.google.com,0
7,2013-11-15 11:40:37,mail.google.com,0
8,2013-11-15 11:40:40,apis.google.com,0
9,2013-11-15 11:41:35,plus.google.com,0


In [3]:
with open('./tools.py', 'r') as f:
    exec(f.read())

In [5]:
path_to_csv_files = os.path.join(PATH_TO_DATA, '3users')

In [6]:
frame_sites, site_freq = prepare_train_set(path_to_csv_files, text=True)

frame_sites.head()

,user_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,,
0,1,vk,oracle,oracle,geo.mozilla.org,oracle,google,accounts.google,mail.google,apis.google,plus.google
1,1,vk,google,google,google,0,0,0,0,0,0
0,2,vk,oracle,football.kulichki.ru,football.kulichki.ru,oracle,0,0,0,0,0
0,3,meduza.io,google,oracle,google,oracle,google,google,mail.google,yandex.ru,meduza.io
1,3,meduza.io,google,oracle,0,0,0,0,0,0,0


In [22]:
path_to_csv_files_10users = os.path.join(PATH_TO_DATA, '10users')
path_to_csv_files_150users = os.path.join(PATH_TO_DATA, '150users')

train_data_10users, site_freq_10users = prepare_train_set(path_to_csv_files_10users, text=True)
train_data_150users, site_freq150users = prepare_train_set(path_to_csv_files_150users, text=True)

In [18]:
train_data_150users.head()

,user_id,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,,
0,1,armmf.adobe,armmf.adobe,armdl.adobe,javadl-esd-secure.oracle,javadl-esd-secure.oracle,oracle,apex.oracle,oracle,gtssl-ocsp.geotrust,gtssl-ocsp.geotrust
1,1,gtglobal-ocsp.geotrust,clients1.google,squid-cache.org,geo.mozilla.org,sb-ssl.google,safebrowsing-cache.google,safebrowsing.clients.google,safebrowsing-cache.google,ocsp.thawte,ocsp.verisign
2,1,ocsp.digicert,safebrowsing.clients.google,safebrowsing-cache.google,safebrowsing.clients.google,safebrowsing-cache.google,safebrowsing-cache.google,javadl-esd-secure.oracle,javadl-esd-secure.oracle,oracle,oracle
3,1,apex.oracle,oracle,oracle,apex.oracle,javadl-esd-secure.oracle,oracle,apex.oracle,javadl-esd-secure.oracle,javadl-esd-secure.oracle,go.microsoft
4,1,windows.microsoft,ajax.microsoft,windows.microsoft,res1.windows.microsoft,res2.windows.microsoft,js.microsoft,js.microsoft,res1.windows.microsoft,bing,go.microsoft


In [23]:
X_10users, y_10users = train_data_10users[['site'+str(r) for r in range(1,10+1)]].values, \
                       train_data_10users.reset_index('session_id')[['user_id']].values
X_150users, y_150users = train_data_150users[['site'+str(r) for r in range(1,10+1)]].values, \
                       train_data_150users.reset_index('session_id')[['user_id']].values

In [14]:
train_data_10users.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), float_format='%d')
train_data_150users.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), float_format='%d')

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy

corpus = X_10users.tolist()
corpus_text = list(map(lambda s: ' '.join(s), corpus))

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus_text)
X_sparse_10users  = X.toarray()


with open(os.path.join(PATH_TO_DATA, 'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy

corpus = X_150users.tolist()
corpus_text = list(map(lambda s: ' '.join(s), corpus))

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus_text)
X_sparse_150users  = X.toarray()


with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(csr_matrix(X_sparse_150users), X150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy

corpus = frame_sites[['site'+str(r) for r in range(1,10+1)]].values.tolist()
corpus_text = list(map(lambda s: ' '.join(s), corpus))

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus_text)
X.toarray()

array([[1, 1, 0, 1, 5, 0, 0, 1, 0, 1, 3, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 1, 0],
       [0, 0, 0, 0, 5, 2, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]])

In [104]:
vectorizer.get_feature_names()fit_transformcorpus_text = map(to_text, corpus)

[u'10', u'11']

In [123]:
corpus_text

['vk oracle oracle geo.mozilla.org oracle google accounts.google mail.google apis.google plus.google',
 'vk google google google 0 0 0 0 0 0',
 'vk oracle football.kulichki.ru football.kulichki.ru oracle 0 0 0 0 0',
 'meduza.io google oracle google oracle google google mail.google yandex.ru meduza.io',
 'meduza.io google oracle 0 0 0 0 0 0 0']

In [102]:
def to_text(l):
    li = map(str, l)
    return ' '.join(li)
    corpus_text = map(to_text, corpus)
corpus_text = map(to_text, corpus)

['4 2 2 9 2 1 7 6 8 10',
 '4 1 1 1 0 0 0 0 0 0',
 '4 2 5 5 2 0 0 0 0 0',
 '3 1 2 1 2 1 1 6 11 3',
 '3 1 2 0 0 0 0 0 0 0']

In [100]:
corpus = frame_sites[['site'+str(r) for r in range(1,10+1)]].values.tolist()

corpus[0]

[4, 2, 2, 9, 2, 1, 7, 6, 8, 10]

In [101]:
def to_text(l):
    li = map(str, l)
    return ' '.join(li)

to_text([4, 2, 2, 9, 2, 1, 7, 6, 8, 10])

'4 2 2 9 2 1 7 6 8 10'

In [105]:
corpus = ['4 2 2 9 2 1 7 6 8 10',
 '4 1 1 1 0 0 0 0 0 0',
 '4 2 5 5 2 0 0 0 0 0',
 '3 1 2 1 2 1 1 6 11 3',
 '3 1 2 0 0 0 0 0 0 0']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

[u'10', u'11']
